
## Traitement de flux structuré avec l'API DataFrame Python
Apache Spark inclut une API de traitement de flux de haut niveau : Structured Streaming.
<br>Dans ce TP, nous jetons un coup d'œil rapide l'utilisation de l'API DataFrame pour construire des applications Structured Streaming.
<br>Dans un premier temps, nous réviserons les éléments vus précédemment pour explorer rapidement les données.
<br>Dans un second temps, nous calculerons des métriques en temps réel (dans notre exemple, ce seront des "running counts" et "windowed counts" sur un flux d'actions horodatées).


Uploader les 50 fichiers du dossier events dans le dossier /FileStore/tables/events. Vérifier qu'ils sont bien présents :

In [0]:
%fs ls /FileStore/tables/events

path,name,size,modificationTime
dbfs:/FileStore/tables/events/file_0.json,file_0.json,72530,1710962067000
dbfs:/FileStore/tables/events/file_1.json,file_1.json,72961,1710962068000
dbfs:/FileStore/tables/events/file_10.json,file_10.json,73025,1710962071000
dbfs:/FileStore/tables/events/file_11.json,file_11.json,72999,1710962071000
dbfs:/FileStore/tables/events/file_12.json,file_12.json,72987,1710962072000
dbfs:/FileStore/tables/events/file_13.json,file_13.json,73006,1710962072000
dbfs:/FileStore/tables/events/file_14.json,file_14.json,73003,1710962072000
dbfs:/FileStore/tables/events/file_15.json,file_15.json,73007,1710962072000
dbfs:/FileStore/tables/events/file_16.json,file_16.json,72978,1710962073000
dbfs:/FileStore/tables/events/file_17.json,file_17.json,73008,1710962073000


Exécuter la commande suivante pour voir rapidement ce que les fichiers contiennent à partir de l'exemple du fichier "file_0.json".

In [0]:
%fs head /FileStore/tables/events/file_0.json

[Truncated to first 65536 bytes]
{"time":1469501107,"action":"Open"}
{"time":1469501147,"action":"Open"}
{"time":1469501202,"action":"Open"}
{"time":1469501219,"action":"Open"}
{"time":1469501225,"action":"Open"}
{"time":1469501234,"action":"Open"}
{"time":1469501245,"action":"Open"}
{"time":1469501246,"action":"Open"}
{"time":1469501248,"action":"Open"}
{"time":1469501256,"action":"Open"}
{"time":1469501264,"action":"Open"}
{"time":1469501266,"action":"Open"}
{"time":1469501267,"action":"Open"}
{"time":1469501269,"action":"Open"}
{"time":1469501271,"action":"Open"}
{"time":1469501282,"action":"Open"}
{"time":1469501285,"action":"Open"}
{"time":1469501291,"action":"Open"}
{"time":1469501297,"action":"Open"}
{"time":1469501303,"action":"Open"}
{"time":1469501322,"action":"Open"}
{"time":1469501335,"action":"Open"}
{"time":1469501344,"action":"Open"}
{"time":1469501346,"action":"Open"}
{"time":1469501349,"action":"Open"}
{"time":1469501357,"action":"Open"}
{"time":1469501366,"action":"Open"}
{"time":1469501371,"action":"Open"}
{"time":1469501375,"action":"Open"}
{"time":1469501375,"action":"Open"}
{"time":1469501381,"action":"Open"}
{"time":1469501392,"action":"Open"}
{"time":1469501402,"action":"Open"}
{"time":1469501407,"action":"Open"}
{"time":1469501410,"action":"Open"}
{"time":1469501420,"action":"Open"}
{"time":1469501424,"action":"Open"}
{"time":1469501438,"action":"Open"}
{"time":1469501442,"action":"Close"}
{"time":1469501462,"action":"Open"}
{"time":1469501480,"action":"Open"}
{"time":1469501488,"action":"Open"}
{"time":1469501489,"action":"Open"}
{"time":1469501491,"action":"Open"}
{"time":1469501503,"action":"Open"}
{"time":1469501505,"action":"Open"}
{"time":1469501509,"action":"Open"}
{"time":1469501513,"action":"Open"}
{"time":1469501517,"action":"Open"}
{"time":1469501520,"action":"Open"}
{"time":1469501525,"action":"Open"}
{"time":1469501533,"action":"Open"}
{"time":1469501539,"action":"Open"}
{"time":1469501540,"action":"Open"}
{"time":1469501541,"action":"Open"}
{"time":1469501543,"action":"Open"}
{"time":1469501544,"action":"Open"}
{"time":1469501545,"action":"Close"}
{"time":1469501545,"action":"Open"}
{"time":1469501547,"action":"Open"}
{"time":1469501552,"action":"Open"}
{"time":1469501557,"action":"Open"}
{"time":1469501559,"action":"Open"}
{"time":1469501560,"action":"Open"}
{"time":1469501560,"action":"Open"}
{"time":1469501565,"action":"Open"}
{"time":1469501566,"action":"Open"}
{"time":1469501574,"action":"Open"}
{"time":1469501575,"action":"Open"}
{"time":1469501575,"action":"Open"}
{"time":1469501578,"action":"Open"}
{"time":1469501581,"action":"Open"}
{"time":1469501584,"action":"Open"}
{"time":1469501600,"action":"Open"}
{"time":1469501601,"action":"Open"}
{"time":1469501603,"action":"Open"}
{"time":1469501610,"action":"Open"}
{"time":1469501620,"action":"Open"}
{"time":1469501621,"action":"Open"}
{"time":1469501625,"action":"Open"}
{"time":1469501625,"action":"Close"}
{"time":1469501626,"action":"Open"}
{"time":1469501631,"action":"Open"}
{"time":1469501632,"action":"Open"}
{"time":1469501632,"action":"Open"}
{"time":1469501638,"action":"Open"}
{"time":1469501643,"action":"Open"}
{"time":1469501646,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501663,"action":"Open"}
{"time":1469501667,"action":"Open"}
{"time":1469501674,"action":"Open"}
{"time":1469501675,"action":"Open"}
{"time":1469501678,"action":"Close"}
{"time":1469501680,"action":"Open"}
{"time":1469501685,"action":"Open"}
{"time":1469501686,"action":"Open"}
{"time":1469501689,"action":"Open"}
{"time":1469501691,"action":"Open"}
{"time":1469501694,"action":"Open"}
{"time":1469501696,"action":"Close"}
{"time":1469501702,"action":"Open"}
{"time":1469501703,"action":"Open"}
{"time":1469501704,"action":"Open"}
{"time":1469501706,"action":"Open"}
{"time":1469501706,"action":"Open"}
{"time":1469501710,"action":"Open"}
{"time":1469501715,"action":"Open"}
{"

Chaque ligne du fichier contient un enregistrement json avec deux champs : `time` and `action`. Nous allons analyser ces fichiers de manière interactive.

## Traitement par lots/interactif
La première étape pour tenter de traiter les données est de les interroger de manière interactive.
Nous allons définir un DataFrame statique sur les fichiers.

In [0]:
input_path = "/FileStore/tables/events"

Pour changer (et en pratique accélérer le processus), définir le schéma (au lieu de demander à Spark de l'inférer).

In [0]:
from pyspark.sql.types import TimestampType, StringType, StructType, StructField

json_schema = StructType(
  [
    StructField("time", TimestampType(), True),
    StructField("action", StringType(), True),
  ]
)

static_in_df = spark.read.schema(json_schema).json(input_path)
display(static_in_df)

time,action
2016-07-28T04:19:28Z,Close
2016-07-28T04:19:28Z,Close
2016-07-28T04:19:29Z,Open
2016-07-28T04:19:31Z,Close
2016-07-28T04:19:31Z,Open
2016-07-28T04:19:31Z,Open
2016-07-28T04:19:32Z,Close
2016-07-28T04:19:33Z,Close
2016-07-28T04:19:35Z,Close
2016-07-28T04:19:36Z,Open


Calculer le nombre d'actions "open" et "close" dans des fenêtres d'une heure.
<br>Indication : utiliser un GROUP BY sur la colonne `action` et des fenêtres (windows) sur la colonne `time`.
<br>Réaliser cette tâche en DSL.
<br>Mettre en cache (en mémoire) le DataFrame obtenu.

In [0]:
from pyspark.sql.functions import window

static_counts_df = static_in_df.groupBy(
  static_in_df["action"],
  window(static_in_df["time"], "1 hour"),
).count()

static_counts_df.cache()

DataFrame[action: string, window: struct<start:timestamp,end:timestamp>, count: bigint]

In [0]:
display(static_counts_df)

action,window,count
Close,"List(2016-07-26T13:00:00Z, 2016-07-26T14:00:00Z)",1028
Open,"List(2016-07-26T18:00:00Z, 2016-07-26T19:00:00Z)",1004
Close,"List(2016-07-27T02:00:00Z, 2016-07-27T03:00:00Z)",971
Open,"List(2016-07-27T04:00:00Z, 2016-07-27T05:00:00Z)",995
Open,"List(2016-07-27T05:00:00Z, 2016-07-27T06:00:00Z)",986
Open,"List(2016-07-26T05:00:00Z, 2016-07-26T06:00:00Z)",1000
Open,"List(2016-07-26T11:00:00Z, 2016-07-26T12:00:00Z)",991
Close,"List(2016-07-26T06:00:00Z, 2016-07-26T07:00:00Z)",1011
Close,"List(2016-07-27T05:00:00Z, 2016-07-27T06:00:00Z)",987
Open,"List(2016-07-26T10:00:00Z, 2016-07-26T11:00:00Z)",1007


<br>Déterminer le nombre total d'actions "open" et d'actions "close" (sur l'ensemble de la période).
<br>Réaliser cette tâche en SQL.

In [0]:
static_counts_df.createOrReplaceTempView("static_counts_view")

total_df = spark.sql(
  """
  SELECT action, sum(count) as total_count
  FROM static_counts_view
  GROUP BY action
  """
)

total_df.show()

+------+-----------+
|action|total_count|
+------+-----------+
| Close|      50000|
|  Open|      50000|
+------+-----------+



Dans un notebook Databricks, il est possible d'exécuter une commande sql en ligne de commande avec la commande %sql, et d'obtenir un visuel automatique.
<br> L'essayer sur l'instruction précédente.

In [0]:
# Votre code ici

L'utilisation des windows se fait de la manière suivante en SQL. Exécuter la commande suivante pour obtenir un visuel des actions dans le temps.

In [0]:
%sql

select action, date_format(window.end, "MMM-dd HH:mm") as time, count from static_counts_view order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,815
Open,Jul-26 05:00,999
Close,Jul-26 06:00,1003
Open,Jul-26 06:00,1000
Close,Jul-26 07:00,1011
Open,Jul-26 07:00,993


Vous devriez remarquer que les actions de fermeture suivent les actions d'ouverture correspondantes (il y a plus d'"open" au début et plus de "close" à la fin).

### Traitement de flux (stream processing)
Maintenant que nous avons analysé les données de manière interactive, nous allons créer une requête de traitement de flux qui se met à jour en continu, à mesure que les données arrivent.
<br>Étant donné que nous avons simplement un ensemble statique de fichiers, nous allons émuler un flux à partir d'eux en lisant un fichier à la fois, dans l'ordre chronologique de leur création.
<br> A noter que le code que nous allons écrire est à peu près le même que précédemment. Merci l'API de Spark !

Adapter le code créé précédemment pour calculer le nombre d'actions "open" et "close", en utilisant readStream à la place de read. Utiliser l'option maxFilesPerTrigger afin de spécifier qu'on ne prend qu'un ficher à la fois (les options doivent être placées avant le chargement effectif à partir de la méthode json).

In [0]:
streaming_in_df = spark.readStream.schema(json_schema).option("maxFilesPerTrigger", 1).json(input_path)

streaming_counts_df = streaming_in_df.groupBy(
  streaming_in_df["action"],
  window(streaming_in_df["time"], "1 hour"),
).count()

En fouillant (avec le nez fin) la documentation des DataFrames (https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.DataFrame.html?highlight=dataframe#pyspark.sql.DataFrame), vérifier si le DataFrame que nous venons de créer est bien un DataFrame de streaming.

In [0]:
static_in_df.isStreaming

False

<br>Vous pouvez démarrer le calcul en continu en définissant la destination (le "sink") et en le démarrant.
<br>Dans notre cas, nous voulons interroger de manière interactive les comptages (mêmes requêtes que ci-dessus), nous allons donc définir l'ensemble complet des comptages d'une heure dans une table en mémoire (pour cela il faudra utiliser format avec l'option "memory").
<br>Utiliser la fonction writeStream pour démarrer la requête.
<br>Donner un nom à la table in-memory, afin de pouvoir y faire référence dans la suite. Pour cela, utiliser queryName.
<br>Choisir un outputMode "complete".

In [0]:
query = streaming_counts_df.writeStream.format("memory").queryName("counts").outputMode("complete").start()

`query` est une référence à la requête de streaming qui s'exécute en arrière-plan. Cette requête récupère continuellement les fichiers et met à jour les comptages fenêtrés. A noter que nous ne faisons que simuler une arrivée continue de fichiers, grâce à l'option maxFilesPerTrigger. Nous n'avons pas précisé d'intervalle de temps entre chaque trigger (il est possible de le paramétrer).

Remarquer le statut de la requête dans la cellule ci-dessus (la barre de progression montre que la requête est active).
<br> Développer `> counts` : on y trouve le nombre de fichiers traités.

Attendre un peu (physiquement ou en utilisant la fonction sleep de la bibliothèque time) puis lancer la requête sql permettant de visualiser les comptages dans le temps sur le DataFrame de streaming. Ne pas oublier que nous avons donné un nom particulier à la table in-memory.

In [0]:
%sql

select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,815
Open,Jul-26 05:00,999
Close,Jul-26 06:00,1003
Open,Jul-26 06:00,1000
Close,Jul-26 07:00,1011
Open,Jul-26 07:00,993


Continuer à exécuter la requête de manière itérative.

In [0]:
# Votre code- (éventuel) ici

Visualiser le nombre total d'actions "open" et "close".

In [0]:
%sql

select action, sum(count) as total_count from counts group by action

action,total_count
Open,50000
Close,50000


Si vous exécutez la requête ci-dessus de manière répétée, vous constaterez que le nombre d'actions "open" est bien en permanence plus élevé que le nombre d'actions "close", comme anticipable dans un flux de données où une fermeture apparaît toujours après l'ouverture qui lui correspondant. Cela montre que Structured Streaming garantit l'"intégrité du préfixe" (prefix integrity).

Arrêter la requête en exécutant query.stop(). Vérifier le statut de la cellule correspondante à la requête.
<br>Ici bien entendu, nous n'avons que quelques fichiers, et il est possible qu'au moment où vous arrivez à cette cellule, ils soient tous consommés. Dans ce cas, il n'y a plus de mise à jour des comptages. Mais il faut quand même arrêter notre requête proprement !

In [0]:
query.stop()